In [1]:
import os
from google import genai
from google.genai import types
from google.genai.types import FunctionDeclaration, FunctionResponse

import random

In [2]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

In [3]:
get_weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to get the weather for."
            }
        },
        "required": ["city"]
    }
}

In [4]:
tools: list[FunctionDeclaration] = [get_weather_tool]
# tools = []

In [5]:


def gemini_config(tools: list[FunctionDeclaration] = []) -> types.GenerateContentConfig:
    if not tools:
        return types.GenerateContentConfig()
    gemini_tools = types.Tool(function_declarations=tools)
    config = types.GenerateContentConfig(tools=[gemini_tools])
    return config

def llm(prompt: str, tools: list[FunctionDeclaration] = []) -> types.GenerateContentResponse:
    gemini_key = os.getenv('GEMINI_API_KEY')
    client = genai.Client(api_key=gemini_key)
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite-preview-06-17",
        contents=prompt,
        config=gemini_config(tools=tools),
    )
    return response


In [6]:
question = "What's the weather like in Berlin?"

In [7]:
# llm_response = llm(
#     prompt=question,
#     tools=tools
# )

In [8]:
# llm_response.text

In [9]:
# q1 = "what's the weather like in Odesa?"
llm_response = llm(
    prompt=question,
    tools=tools
)
llm_response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            )
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash-lite-preview-06-17',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=15,
    prompt_token_count=61,
    prompt_tokens_details=[
      ModalityTokenCount(
        modality=<MediaModality.TEXT: 'TEXT'>,
        token_count=61
      ),
    ],
    total_token_count=76
  )
)

In [10]:
llm_response.candidates[0].content.parts[0].function_call

FunctionCall(
  args={
    'city': 'Berlin'
  },
  name='get_weather'
)

In [11]:
tool_call = llm_response.candidates[0].content.parts[0].function_call

if tool_call.name == "get_weather":
    result = get_weather(**tool_call.args)
    print(f"Function execution result: {result}")


Function execution result: 20.0


In [12]:
function_response_part = types.Part.from_function_response(
    name=tool_call.name,
    response={"result": result},
)

contents = [
    # types.Content(role="user", parts=[types.Part.from_text(question)]),
    # types.Content(role="assistant", parts=[llm_response.candidates[0].content]),
    types.Content(role="user", parts=[types.Part(text=question)])
]
# Append function call and result of the function execution to contents
contents.append(llm_response.candidates[0].content) # Append the content from the model's response.
contents.append(types.Content(role="user", parts=[function_response_part])) # Append the function response

final_response = llm(
    prompt=contents,
    tools=tools
)

print(final_response)
print(final_response.text)

sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text='The weather in Berlin is 20 degrees.'
      ),
    ],
    role='model'
  ),
  finish_reason=<FinishReason.STOP: 'STOP'>,
  index=0
)] create_time=None response_id=None model_version='gemini-2.5-flash-lite-preview-06-17' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=10,
  prompt_token_count=92,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=92
    ),
  ],
  total_token_count=102
) automatic_function_calling_history=[] parsed=None
The weather in Berlin is 20 degrees.


### Q2. Adding another tool

In [13]:
### Q2. Adding another tool

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'


In [20]:
set_weather_tool = {
    "name": "set_weather",
    "description": "Set the weather for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city to set the weather for."
            },
            "temp": {
                "type": "number",
                "description": "The temperature to set for the city."
            }
        },
        "required": ["city", "temp"]
    }
}

In [25]:

from chat_assistant_gemini import ToolManager, ChatUserInterface, AIConversationAssistant

tool_manager = ToolManager()
tool_manager.register_tool(
    set_weather, 
    set_weather_tool
    # FunctionDeclaration(
    #         name="set_weather",
    #         description="Set the weather for a given city.",
    #         parameters={
    #             "type": "object",
    #             "properties": {
    #                 "city": {"type": "string", 
    #                          "description": "The name of the city to set the weather for, e.g. Odesa, UA"
    #                 },
    #                 "temp": {
    #                     "type": "number",
    #                     "description": "The temperature to set for the city."
    #                 }
    #             },
    #             "required": ["city", "temp"]
    #         },
    # )
)

get_weather_tool = {
        "name":"get_weather",
        "description":"Get the current weather for a given city.",
        "parameters":{
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The name of the city to get the weather for."
                },
            },
            "required": ["city"]
        }
}

tool_manager.register_tool(
    get_weather,
    get_weather_tool
    # FunctionDeclaration(
    #     name="get_weather",
    #     description="Get the current weather for a given city.",
    #     parameters={
    #         "type": "object",
    #         "properties": {
    #             "city": {
    #                 "type": "string",
    #                 "description": "The name of the city to get the weather for."
    #             }
    #         },
    #         "required": ["city"]
    #     },
    # )
)


ui = ChatUserInterface()

Registered tool: set_weather
Description: {'name': 'set_weather', 'description': 'Set the weather for a given city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to set the weather for.'}, 'temp': {'type': 'number', 'description': 'The temperature to set for the city.'}}, 'required': ['city', 'temp']}}
Registered tool: get_weather
Description: {'name': 'get_weather', 'description': 'Get the current weather for a given city.', 'parameters': {'type': 'object', 'properties': {'city': {'type': 'string', 'description': 'The name of the city to get the weather for.'}}, 'required': ['city']}}


In [26]:
tool_manager.functions

{'set_weather': <function __main__.set_weather(city: str, temp: float) -> None>,
 'get_weather': <function __main__.get_weather(city: str) -> float>}

In [27]:
tool_manager.get_tool_descriptions()

[{'name': 'set_weather',
  'description': 'Set the weather for a given city.',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The name of the city to set the weather for.'},
    'temp': {'type': 'number',
     'description': 'The temperature to set for the city.'}},
   'required': ['city', 'temp']}},
 {'name': 'get_weather',
  'description': 'Get the current weather for a given city.',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The name of the city to get the weather for.'}},
   'required': ['city']}}]

In [28]:
developer_prompt = "You are a helpful assistant that can answer questions and use tools. If the user asks about weather, use the `get_weather` tool. If the user asks about set weather, use the `set_weather` tool."

assistant = AIConversationAssistant(
    tool_manager=tool_manager,
    developer_prompt=developer_prompt,
    ui_interface=ui,
    model_name='gemini-2.5-flash-lite-preview-06-17'
)


assistant.start_chat()

KeyError: 'object'